In [336]:
from pprint import pprint
import pandas as pd
import pyodbc
import math
server='tarpley.database.windows.net'
database='marketr'
username='tristan'
password='Fiverrtemp!'
driver='{ODBC Driver 17 for SQL Server}'

connStr = 'DRIVER='+driver+';SERVER='+server+';PORT=1443;DATABASE='+database+';UID='+username+';PWD='+ password

### 1) Base mix

In [350]:
base_mix = {
    'awareness': .55,
    'evaluation': .25,
    'conversion': .2
}

### 2) AEC Adjustment Table

In [351]:
def adjustments(brand_strength, growth_needs, competitiveness):
    adjustments = {}
    considerations = ['brand_strength', 'growth_needs', 'competitiveness']
    levels = ['low', 'medium', 'high']
    stages = ['awareness', 'evaluation', 'conversion']
    
    brand_values = [
        [.1, -.05, -.05],
        [0,0,0],
        [-.1, .05, .05]
    ]
    growth_values = [
        [-.1, .03, .07],
        [0,0,0],
        [.07, -.04, -.03]
    ]
    competitiveness_values = [
        [-.1, .03, .07],
        [0,0,0],
        [0, .05, -.05]
    ]
    
    def pack_values():
        for consideration in enumerate(considerations):
            adjustments[consideration[1]] = {}
            for level in enumerate(levels):
                adjustments[consideration[1]][level[1]] = {}
                for stage in enumerate(stages):
                    if consideration[1] == 'brand_strength':
                        lookup = brand_values[level[0]][stage[0]]
                    elif consideration[1] == 'growth_needs':
                        lookup = growth_values[level[0]][stage[0]]
                    elif consideration[1] == 'competitiveness':
                        lookup = competitiveness_values[level[0]][stage[0]]
                    
                    adjustments[consideration[1]][level[1]][stage[1]] = lookup
                 
    pack_values()
                
    awareness = (
          adjustments['brand_strength'][brand_strength]['awareness']
        + adjustments['growth_needs'][growth_needs]['awareness']
        + adjustments['competitiveness'][competitiveness]['awareness']
    )
    evaluation = (
          adjustments['brand_strength'][brand_strength]['evaluation']
        + adjustments['growth_needs'][growth_needs]['evaluation']
        + adjustments['competitiveness'][competitiveness]['evaluation']
    )
    conversion = (
          adjustments['brand_strength'][brand_strength]['conversion']
        + adjustments['growth_needs'][growth_needs]['conversion']
        + adjustments['competitiveness'][competitiveness]['conversion']
    )
    return awareness, evaluation, conversion

### 3) # of tactics to select

In [367]:
def num_tactics(budget):
    if budget <= 1000:
        tactics = 3
    elif budget > 1000 and budget <= 3000:
        tactics = 4
    elif budget > 3000 and budget <= 5000:
        tactics = 6
    elif budget > 5000 and budget <= 10000:
        tactics = 8
    elif budget > 10000:
        tactics = 10
        
    return math.floor(tactics)

### 4) Business tag lookup & tactic selection

In [368]:
def get_tags(customer_id):
    db = pyodbc.connect(connStr)
    cursor = db.cursor()
    cursor.execute("exec get_spend_tags @customer_id = ?", (customer_id,))
    tags_db = cursor.fetchall()

    tags = []
    for tag in tags_db:
        tags.append({
            'tag': tag[0],
            'tactic': tag[1],
            'category': tag[2],
            'priority_scale': tag[3]
        })

    return pd.DataFrame(tags).sort_values(by=['priority_scale'], ascending=False)

# Calculator

In [474]:
def allocation(revenue, budget, brand_strength, growth_needs, competitiveness, biz_type, biz_model):
    base_table = base_mix
    awareness_adj, evaluation_adj, conversion_adj = adjustments(brand_strength, growth_needs, competitiveness)
    
    awareness, evaluation, conversion = (
        (base_table['awareness'] + awareness_adj),
        (base_table['evaluation'] + evaluation_adj),
        (base_table['conversion'] + conversion_adj)
    )
    awareness_spend, evaluation_spend, conversion_spend = (
        awareness * budget,
        evaluation * budget,
        conversion * budget
    )
    
    tactics = num_tactics(budget)
    awareness_tactics, evaluation_tactics, conversion_tactics = (
        math.floor(base_table['awareness'] * tactics),
        round(base_table['evaluation'] * tactics),
        round(base_table['conversion'] * tactics)
    )
    
    tags = get_tags(198)
    awareness = tags[tags['category'] == 'Awareness'][:awareness_tactics]
    evaluation = tags[tags['category'] == 'Evaluation'][:evaluation_tactics]
    conversion = tags[tags['category'] == 'Conversion'][:conversion_tactics]
    
    def tactic_spend(priority, cat_spend, sum):
        return round(cat_spend*priority/sum)
    
    for tup in [(awareness, awareness_spend), (evaluation, evaluation_spend), (conversion, conversion_spend)]:
        df = tup[0]
        spend = tup[1]
        df['spend_per_tactic'] = df['priority_scale'].apply(
            lambda x: tactic_spend(x, spend, df['priority_scale'].sum())
        )
        
        
        
    return (awareness, evaluation, conversion)

In [475]:
revenue = 500000
budget = 3875
brand_strength = 'low'
growth_needs = 'medium'
competitiveness = 'medium'
biz_type = 'b2b'
biz_model = 'SaaS'

In [471]:
allocation = allocation(revenue, budget, brand_strength, growth_needs, competitiveness, biz_type, biz_model)